In [ ]:
import pathlib
from pprint import pprint
from matplotlib import pyplot as plt
import numpy as np
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

In [3]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent.parent.parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox" / "DC"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

In [4]:
benchmark4 = PowerGridBenchmark(benchmark_name="Benchmark4",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH,
                               )

In [ ]:
benchmark4.config.get_option("eval_dict")

In [6]:
from lips.augmented_simulators.torch_models.gnn import TorchGCN
from lips.augmented_simulators.torch_simulator import TorchSimulator
#from lips.dataset.scaler import StandardScaler

In [7]:
torch_sim = TorchSimulator(name="torch_gcn",
                           model=TorchGCN,
                           scaler=None,
                           log_path=LOG_PATH,
                           device="cpu", # use "cpu" if you don't have a GPU available on your machine
                           seed=42,
                           bench_config_path=BENCH_CONFIG_PATH,
                           bench_config_name="Benchmark4",
                           sim_config_path=SIM_CONFIG_PATH / "torch_gnn.ini",
                           sim_config_name="GCN_CONFIG_RECOMMENDED_1" # use the default set of hyper parameters
                          )

In [ ]:
torch_sim.params

### Train the augmented simulator

In [9]:
torch_sim.train(benchmark4.train_dataset, benchmark4.val_dataset, save_path=None, epochs=1, train_batch_size=128)

Batch within epoch (Training): 100%|██████████| 782/782 [01:05<00:00, 12.01it/s]


Train Epoch: 1   Avg_Loss: 131.93107 ['MAELoss: 29.57365']


Epoch 1: 100%|██████████| 1/1 [01:08<00:00, 68.54s/it]


Eval:   Avg_Loss: 86.19561 ['MAELoss: 25.43060']


### Save the trained model

In [20]:
SAVE_PATH = get_path(TRAINED_MODEL_PATH, benchmark4)
torch_sim.save(SAVE_PATH)

### Load the trained model

In [6]:
from lips.augmented_simulators.torch_models.gnn import TorchGCN
from lips.augmented_simulators.torch_simulator import TorchSimulator

torch_sim = TorchSimulator(name="torch_gcn",
                           model=TorchGCN,
                           scaler=None,
                           log_path=LOG_PATH,
                           device="cpu", # use "cpu" if you don't have a GPU available on your machine
                           seed=42,
                           bench_config_path=BENCH_CONFIG_PATH,
                           bench_config_name="Benchmark4",
                           sim_config_path=SIM_CONFIG_PATH / "torch_gnn.ini",
                           sim_config_name="GCN_CONFIG_RECOMMENDED_1", # use the default set of hyper parameters
                           #attr_y=("theta_or", "theta_ex")
                          )

In [7]:
LOAD_PATH = get_path(TRAINED_MODEL_PATH, benchmark4)
torch_sim.restore(path=LOAD_PATH)

### Predict using the trained augmented simulator

In [10]:
metrics = benchmark4.evaluate_simulator(dataset="all", augmented_simulator=torch_sim, save_path=None, save_predictions=False)

Batch (Prediction): 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


In [12]:
metrics["test"]

{'ML': {'MSE_avg': {'theta_or': 0.0,
   'theta_ex': 0.0,
   'p_or': 481.1911926269531,
   'p_ex': 481.1911926269531},
  'MAE_avg': {'theta_or': 0.0,
   'theta_ex': 0.0,
   'p_or': 14.482661247253418,
   'p_ex': 14.482661247253418},
  'MAPE_avg': {'theta_or': 0.0,
   'theta_ex': 0.0,
   'p_or': 2084222806261760.0,
   'p_ex': 2084222806261760.0},
  'MAPE_90_avg': {'theta_or': nan,
   'theta_ex': 0.0,
   'p_or': 0.5942630115912368,
   'p_ex': 0.5942630115912368},
  'TIME_INF': 20.475355890986975},
 'Physics': {'LOSS_POS': {}},
 'IndRed': {'TIME_INF': 22.10974759599776}}